In [123]:
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import torch.nn.functional as F

In [124]:
III_label = pd.read_csv('/data5/youngju/proj2/train/labels/mimiciii_labels.csv')
IV_label = pd.read_csv('/data5/youngju/proj2/train/labels/mimiciv_labels.csv')
EICU_label = pd.read_csv('/data5/youngju/proj2/train/labels/eicu_labels.csv')

In [125]:
EICU_label.columns

Index(['patientunitstayid', 'labels'], dtype='object')

In [126]:
inter = pd.concat([III_label, IV_label], axis=0).sort_values('labels')
df = pd.concat([inter, EICU_label], axis=0).sort_values('labels')
df = df.reset_index(drop=True)
df

,ICUSTAY_ID,labels,stay_id,patientunitstayid
0,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,3012439.0
1,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,3111078.0
2,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,3104814.0
3,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,3052456.0
4,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,3007416.0
...,...,...,...,...
182992,NaN,"[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...",3043765.0,NaN
182993,3006763.0,"[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, ...",NaN,NaN
182994,NaN,"[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, ...",3032563.0,NaN
182995,NaN,"[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, ...",3012663.0,NaN


In [127]:
def hist_num_class(df, task):
    class_list = []
    for i in range(len(df)):
        class_list.append(eval(df['labels'][i])[task])
    #plt.hist(class_list)
    #plt.show()
    return class_list

In [128]:
for i in tqdm(range(28)):
    globals()[f'task_{i}'.format(i)] = hist_num_class(df, i)

100%|██████████| 28/28 [02:16<00:00,  4.87s/it]


In [182]:
def make_weight_binary(task_list):
    task_dct = Counter(task_list)
    task_dct = dict(sorted(task_dct.items()))
    #inv_list = []
    return task_dct[0]/task_dct[1]

In [183]:
for i in range(0,22):
    print(f'{i}번째 task에서의 positive weight:', make_weight_binary(globals()[f'task_{i}'.format(i)]))

0번째 task에서의 positive weight: 122.39649359406609
1번째 task에서의 positive weight: 11.715188993885493
2번째 task에서의 positive weight: 12.792357552004823
3번째 task에서의 positive weight: 3.278329787482758
4번째 task에서의 positive weight: 5.132196233496415
5번째 task에서의 positive weight: 1.0147419876911559
6번째 task에서의 positive weight: 2.519037729318103
7번째 task에서의 positive weight: 2.4699926048125604
8번째 task에서의 positive weight: 2.6922099146540766
9번째 task에서의 positive weight: 0.44384305247627087
10번째 task에서의 positive weight: 1.2542961676336895
11번째 task에서의 positive weight: 2.135550529454097
12번째 task에서의 positive weight: 1.8028764416670495
13번째 task에서의 positive weight: 247.2998643147897
14번째 task에서의 positive weight: 15.08623417721519
15번째 task에서의 positive weight: 6.097858971375378
16번째 task에서의 positive weight: 51.43467048710602
17번째 task에서의 positive weight: 2.4604126089669647
18번째 task에서의 positive weight: 2.7652928952079177
19번째 task에서의 positive weight: 2.2490057524323555
20번째 task에서의 positive weight: 1.66973

In [175]:
def make_weight_multi(task_list):
    task_dct = Counter(task_list)
    task_dct = dict(sorted(task_dct.items()))
    del task_dct[-1]
    inv_list = []
    for i in range(len(task_dct.keys())):
        inv_list.append(sum(np.array(list(task_dct.values()))) / (task_dct[i]+1e-16))
    return inv_list / sum(inv_list)

In [176]:
for i in range(22,28):
    print(f'{i}번째 task에서의 weight:', make_weight_multi(globals()[f'task_{i}'.format(i)]))

22번째 task에서의 weight: [0.02604825 0.36883271 0.23082857 0.10911587 0.18184929 0.08332531]
23번째 task에서의 weight: [1.56384098e-02 7.48118462e-03 1.32822829e-04 2.61799853e-02
 7.59219575e-01 1.91348023e-01]
24번째 task에서의 weight: [0.01799729 0.05513065 0.11623186 0.37119331 0.43944689]
25번째 task에서의 weight: [0.01799812 0.09891459 0.09129632 0.32956744 0.46222352]
26번째 task에서의 weight: [0.0067478  0.02027087 0.03689895 0.14224644 0.79383594]
27번째 task에서의 weight: [0.84437757 0.05745262 0.09816981]


In [187]:
binary_weights = []
for i in range(0,22):
    binary_weights.append([np.around(make_weight_binary(globals()[f'task_{i}'.format(i)]),2)])

In [188]:
binary_weights

[[122.4],
 [11.72],
 [12.79],
 [3.28],
 [5.13],
 [1.01],
 [2.52],
 [2.47],
 [2.69],
 [0.44],
 [1.25],
 [2.14],
 [1.8],
 [247.3],
 [15.09],
 [6.1],
 [51.43],
 [2.46],
 [2.77],
 [2.25],
 [1.67],
 [7.45]]

In [173]:
for sample in binary_weights:
    sample[]sample[0]

[[0.01, 0.99],
 [0.08, 0.92],
 [0.07, 0.93],
 [0.23, 0.77],
 [0.16, 0.84],
 [0.5, 0.5],
 [0.28, 0.72],
 [0.29, 0.71],
 [0.27, 0.73],
 [0.69, 0.31],
 [0.44, 0.56],
 [0.32, 0.68],
 [0.36, 0.64],
 [0.0, 1.0],
 [0.06, 0.94],
 [0.14, 0.86],
 [0.02, 0.98],
 [0.29, 0.71],
 [0.27, 0.73],
 [0.31, 0.69],
 [0.37, 0.63],
 [0.12, 0.88]]

In [178]:
multi_weights = []
for i in range(22,28):
    multi_weights.append(list(np.around(make_weight_multi(globals()[f'task_{i}'.format(i)]),2)))

In [179]:
multi_weights

[[0.03, 0.37, 0.23, 0.11, 0.18, 0.08],
 [0.02, 0.01, 0.0, 0.03, 0.76, 0.19],
 [0.02, 0.06, 0.12, 0.37, 0.44],
 [0.02, 0.1, 0.09, 0.33, 0.46],
 [0.01, 0.02, 0.04, 0.14, 0.79],
 [0.84, 0.06, 0.1]]